# 데모 데이터 생성

In [6]:
import os
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import json
import re
import sys
import time

In [7]:
# msn_train= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_train.csv')
msn_dev= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_dev.csv')

In [8]:
demo_dev = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_dev/news.tsv', delimiter='\t', header=None)
print(demo_dev.shape)
demo_dev.head()

(18723, 8)


,0,1,2,3,4,5,6,7
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
3,N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [9]:
demo_dev_nid = demo_dev.iloc[:,0]
print(len(demo_dev_nid))

demo_devs = msn_dev.loc[msn_dev.iloc[:,6].isin(demo_dev_nid)]
print(demo_devs.shape)
demo_devs.head()

18723
(10534, 13)


,5,nids,Title,Date,Image,Content,0,1,2,3,4,6,7
12,https://assets.msn.com/labs/mind/AABDHTv.html,AABDHTv,"50 Foods You Should Never Eat, According to He...",2019-05-21T21:10:55Z,https://img-s-msn-com.akamaized.net/tenant/amp...,What's the one food you refuse to eat? Whateve...,N22273,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]
19,https://assets.msn.com/labs/mind/AAJfTqo.html,AAJfTqo,Newark Liberty Airport's Terminal One a $2.7 b...,2019-10-23T19:44:51Z,NaN,"Gov. Phil Murphy, Senate President Stephen Swe...",N10616,travel,travelarticle,Newark Liberty Airport's Terminal One a $2.7 b...,"The project, which is the bi-state agency's si...","[{""Label"": ""Newark Liberty International Airpo...",[]
22,https://assets.msn.com/labs/mind/AAJgNxm.html,AAJgNxm,Elijah Cummings to lie in state at US Capitol ...,2019-10-24T09:23:00Z,NaN,WASHINGTON â Rep. Elijah Cummings will lie i...,N21935,news,newspolitics,Elijah Cummings to lie in state at US Capitol ...,"Cummings, a Democrat whose district included s...","[{""Label"": ""Elijah Cummings"", ""Type"": ""P"", ""Wi...","[{""Label"": ""Elijah Cummings"", ""Type"": ""P"", ""Wi..."
24,https://assets.msn.com/labs/mind/AADlunl.html,AADlunl,17 Abandoned Theme Parks to Explore for Thrill...,2019-10-02T11:00:00Z,https://img-s-msn-com.akamaized.net/tenant/amp...,Amusement parks usually conjure up images of s...,N9663,travel,traveltripideas,17 Abandoned Theme Parks to Explore for Thrill...,"Disney, Six Flags, and even the Flintstones ha...","[{""Label"": ""Amusement park"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Six Flags"", ""Type"": ""O"", ""Wikidata..."
30,https://assets.msn.com/labs/mind/AAEw1Zd.html,AAEw1Zd,10 Things You Never Knew About Owls,2019-07-18T14:47:14Z,https://img-s-msn-com.akamaized.net/tenant/amp...,"There's no one ""owl""âin fact there are 250 d...",N16695,lifestyle,lifestylepetsanimals,10 Things You Never Knew About Owls,"With their huge eyes, sharp talons, and swivel...",[],[]


### 추가 수집할 데이터 

In [10]:
dev_nids = []
for nid in demo_dev.iloc[:,0].tolist():
    if nid not in demo_devs.iloc[:,6].tolist():
        dev_nids.append(nid)

print(len(dev_nids))

not_dev = demo_dev.loc[demo_dev.iloc[:,0].isin(dev_nids)]
not_dev.head() 

8189


,0,1,2,3,4,5,6,7
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
5,N21078,entertainment,gaming,Best PS5 games: top PlayStation 5 titles to lo...,Every confirmed or expected PS5 game we can't ...,https://www.msn.com/en-us/entertainment/gaming...,"[{""Label"": ""PlayStation"", ""Type"": ""J"", ""Wikida...",[]
6,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://www.msn.com/en-us/health/nutrition/50-...,[],[]
7,N24180,autos,autosenthusiasts,Trying to Make a Ram 3500 as Quick as a Viper ...,The 2019 Ram 3500's new Cummins diesel has 100...,https://www.msn.com/en-us/autos/autosenthusias...,"[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat...","[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat..."
8,N14522,health,wellness,Instagram Filters with Plastic Surgery-Inspire...,In an effort to combat some of the negative me...,https://www.msn.com/en-us/health/wellness/inst...,"[{""Label"": ""Instagram"", ""Type"": ""W"", ""Wikidata...",[]


In [11]:
df = pd.DataFrame({'urls':not_dev.iloc[:,5].tolist()})
df.to_csv('not_dev.csv', index=False)

In [6]:
url_lst = []
titles = []    
# dates = []
contents = []
images = [] 

urls = not_dev.iloc[:,5].tolist()

print('crawling start.')
start_time = int(time.time())

url_except =[]
num = 0
total = len(urls)
for url in urls:
    # 진행상황 확인 
    num += 1
    if num % 2000 == 0:
        print('{} urls are crawled.\n {} are remained'.format(num, total - num))

    try:
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        # url
        url_lst.append(url)

        # title
        if soup.find('h1').text.strip() is not None:
            title = soup.find('h1').text.strip()
            titles.append(title)
        elif soup.find('header').text.strip() is not None:
            title = soup.find('header').text.strip()
            titles.append(title)
        else:
            titles.append(None)

#         # date
#         try:
#             date = soup.find('time').get('datetime')
#             dates.append(date)
#         except:
#             dates.append(None)

        # content
        if soup.find_all('p') is not None:
            content = []
            for el in soup.find_all('p'):
                content.append(el.get_text()) 
            contents.append(' '.join(content))
        elif soup.find('h2').text.strip() is not None:
            content = soup.find('h2').text.strip()
            contents.append(content)
        else:
            contents.append(url)

        # image
        try:
            image = soup.find('img').get('src')
            images.append(image)
        except:
            images.append('no image')


        # 길이 맞추기 
        if len(url_lst) != num:
            url_lst.append(None)
        if len(titles) != num:
            titles.append(None)
#         if len(dates) != num:
#             dates.append(None)
        if len(contents) != num:
            contents.append(None)
        if len(images) != num:
            images.append(None)

    except:
        url_except.append(url)

print('-------Seconds: %s--------' % (time.time() - start_time))

crawling start.
2000 urls are crawled.
 6189 are remained
4000 urls are crawled.
 4189 are remained
6000 urls are crawled.
 2189 are remained
8000 urls are crawled.
 189 are remained
-------Seconds: 2647.323595046997--------


In [7]:
print('len url_lst', len(url_lst))
print('len titles', len(titles))
print('len dates', len(dates))
print('len contents', len(contents))
print('len images', len(images))

len url_lst 8189
len titles 8189
len dates 8189
len contents 8189
len images 8189


In [2]:
import pandas as pd

In [4]:
demo = pd.read_csv('/Document/SNU_DLab/demo_dev_news.csv')
demo.head() 

FileNotFoundError: [Errno 2] File /Document/SNU_DLab/demo_dev_news.csv does not exist: '/Document/SNU_DLab/demo_dev_news.csv'

In [8]:
df = pd.DataFrame({'url' : url_lst,
                   'Title':titles,
                   'Date': dates, 
                   'Content': contents,
                   'Image': images})


# 데이터 저장   
print('-------df--------')
print(df.shape)
df

-------df--------
(8189, 5)


,url,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/entertainment/gaming...,Best PS5 games: top PlayStation 5 titles to lo...,None,"The Sony PlayStation 5 is coming and, thanks t...",//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/autos/autosenthusias...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/wellness/inst...,Instagram Filters with Plastic Surgery-Inspire...,None,"Instagram filters, especially the stylish ones...",//static-entertainment-eas-s-msn-com.akamaized...
...,...,...,...,...,...
8184,https://www.msn.com/en-us/finance/finance-comp...,Ex-billionaire dragged to court for unpaid $68...,None,Three Chinese banks are suing the brother of A...,//static-entertainment-eas-s-msn-com.akamaized...
8185,https://www.msn.com/en-us/news/world/mexican-c...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
8186,https://www.msn.com/en-us/news/elections-2020-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
8187,https://www.msn.com/en-us/health/healthnews/mo...,"More than 130,000 pounds of ground beef recall...",None,"About 130,464 pounds of raw ground beef produc...",//static-entertainment-eas-s-msn-com.akamaized...


In [10]:
df.loc[df['Title']=='Whoops!']

,5,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/autos/autosenthusias...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
9,https://www.msn.com/en-us/health/weightloss/th...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
19,https://www.msn.com/en-us/lifestyle/lifestyleb...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
...,...,...,...,...,...
8179,https://www.msn.com/en-us/finance/finance-top-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
8180,https://www.msn.com/en-us/health/medical/13-un...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
8182,https://www.msn.com/en-us/news/us/more-than-40...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
8185,https://www.msn.com/en-us/news/world/mexican-c...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...


In [12]:
import numpy as np

for i in range(len(df)):
    if df.loc[i, 'Title'] == 'Whoops!':
        df.loc[i, 'Title'] = np.nan 
        df.loc[i, 'Date'] = np.nan 
        df.loc[i, 'Content'] = np.nan 
        df.loc[i, 'Image'] = np.nan 

df.head()

,5,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN
1,https://www.msn.com/en-us/entertainment/gaming...,Best PS5 games: top PlayStation 5 titles to lo...,None,"The Sony PlayStation 5 is coming and, thanks t...",//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN
3,https://www.msn.com/en-us/autos/autosenthusias...,NaN,NaN,NaN,NaN
4,https://www.msn.com/en-us/health/wellness/inst...,Instagram Filters with Plastic Surgery-Inspire...,None,"Instagram filters, especially the stylish ones...",//static-entertainment-eas-s-msn-com.akamaized...


In [11]:
print('-------url_except-----')
print(len(url_except)) 

-------url_except-----
125


In [13]:
print(demo_devs.shape)
print(df.shape)
print(pd.concat([demo_devs,df]).shape)

(10534, 13)
(8189, 5)
(18723, 14)


In [9]:
df.rename(columns = {'url' : 5}, inplace = True)
df.head()

,5,Title,Date,Content,Image
0,https://www.msn.com/en-us/news/world/the-cost-...,Whoops!,None,This page doesn’t exist or can’t be found. To ...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/entertainment/gaming...,Best PS5 games: top PlayStation 5 titles to lo...,None,"The Sony PlayStation 5 is coming and, thanks t...",//static-entertainment-eas-s-msn-com.akamaized...
2,https://www.msn.com/en-us/health/nutrition/50-...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
3,https://www.msn.com/en-us/autos/autosenthusias...,Whoops!,None,This page is gone. To find something you’ll li...,//static-entertainment-eas-s-msn-com.akamaized...
4,https://www.msn.com/en-us/health/wellness/inst...,Instagram Filters with Plastic Surgery-Inspire...,None,"Instagram filters, especially the stylish ones...",//static-entertainment-eas-s-msn-com.akamaized...


In [14]:
demo_dev = pd.merge(not_dev, df, how='inner',on= 5)
print(demo_dev.shape)
demo_dev.head()

(8186, 12)


,0,1,2,3,4,5,6,7,Title,Date,Content,Image
0,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",NaN,NaN,NaN,NaN
1,N21078,entertainment,gaming,Best PS5 games: top PlayStation 5 titles to lo...,Every confirmed or expected PS5 game we can't ...,https://www.msn.com/en-us/entertainment/gaming...,"[{""Label"": ""PlayStation"", ""Type"": ""J"", ""Wikida...",[],Best PS5 games: top PlayStation 5 titles to lo...,None,"The Sony PlayStation 5 is coming and, thanks t...",//static-entertainment-eas-s-msn-com.akamaized...
2,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://www.msn.com/en-us/health/nutrition/50-...,[],[],NaN,NaN,NaN,NaN
3,N24180,autos,autosenthusiasts,Trying to Make a Ram 3500 as Quick as a Viper ...,The 2019 Ram 3500's new Cummins diesel has 100...,https://www.msn.com/en-us/autos/autosenthusias...,"[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat...","[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat...",NaN,NaN,NaN,NaN
4,N14522,health,wellness,Instagram Filters with Plastic Surgery-Inspire...,In an effort to combat some of the negative me...,https://www.msn.com/en-us/health/wellness/inst...,"[{""Label"": ""Instagram"", ""Type"": ""W"", ""Wikidata...",[],Instagram Filters with Plastic Surgery-Inspire...,None,"Instagram filters, especially the stylish ones...",//static-entertainment-eas-s-msn-com.akamaized...


In [15]:
demo_devs.columns.tolist() 

['5',
 'nids',
 'Title',
 'Date',
 'Image',
 'Content',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7']

In [16]:
demo_dev = demo_dev.iloc[:,[5,8,9,11,10,0,1,2,3,4,6,7]]
demo_dev.columns.tolist() 

[5, 'Title', 'Date', 'Image', 'Content', 0, 1, 2, 3, 4, 6, 7]

In [17]:
demo_dev.rename(columns = {5 : '5'}, inplace = True)
demo_dev.rename(columns = {0 : '0'}, inplace = True)
demo_dev.rename(columns = {1 : '1'}, inplace = True)
demo_dev.rename(columns = {2 : '2'}, inplace = True)
demo_dev.rename(columns = {3 : '3'}, inplace = True)
demo_dev.rename(columns = {4 : '4'}, inplace = True)
demo_dev.rename(columns = {6 : '6'}, inplace = True)
demo_dev.rename(columns = {7 : '7'}, inplace = True)

In [18]:
demo_devs = demo_devs.drop(['nids'], axis=1)
demo_devs.columns.tolist()

['5', 'Title', 'Date', 'Image', 'Content', '0', '1', '2', '3', '4', '6', '7']

In [19]:
demo = pd.concat([demo_dev, demo_devs])
print(demo.shape)
demo.head()

(18720, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
1,https://www.msn.com/en-us/entertainment/gaming...,Best PS5 games: top PlayStation 5 titles to lo...,None,//static-entertainment-eas-s-msn-com.akamaized...,"The Sony PlayStation 5 is coming and, thanks t...",N21078,entertainment,gaming,Best PS5 games: top PlayStation 5 titles to lo...,Every confirmed or expected PS5 game we can't ...,"[{""Label"": ""PlayStation"", ""Type"": ""J"", ""Wikida...",[]
2,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]
3,https://www.msn.com/en-us/autos/autosenthusias...,NaN,NaN,NaN,NaN,N24180,autos,autosenthusiasts,Trying to Make a Ram 3500 as Quick as a Viper ...,The 2019 Ram 3500's new Cummins diesel has 100...,"[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat...","[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat..."
4,https://www.msn.com/en-us/health/wellness/inst...,Instagram Filters with Plastic Surgery-Inspire...,None,//static-entertainment-eas-s-msn-com.akamaized...,"Instagram filters, especially the stylish ones...",N14522,health,wellness,Instagram Filters with Plastic Surgery-Inspire...,In an effort to combat some of the negative me...,"[{""Label"": ""Instagram"", ""Type"": ""W"", ""Wikidata...",[]


In [20]:
demo.to_csv('demo_dev_news.csv', index=False)